In [8]:
import pandas as pd
import numpy as np
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from sklearn.preprocessing import StandardScaler
import random
from sklearn.cluster import KMeans

In [12]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
link = 'hhttps://docs.google.com/spreadsheets/d/1baKBXDs_PiIIYyUFmq0UOCBcMwq6DWji/edit?usp=sharing&ouid=100934670427194582945&rtpof=true&sd=true'
id = '1baKBXDs_PiIIYyUFmq0UOCBcMwq6DWji'
print (id) 

1baKBXDs_PiIIYyUFmq0UOCBcMwq6DWji


In [13]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('sales_df_completed_uc.xlsx')  
st_df1 = pd.read_excel('sales_df_completed_uc.xlsx')

In [14]:
st_df2= st_df1[st_df1['respond_to_discount'] == 1]

In [15]:
columns_to_drop = [ 'cust_id','E Mail','cluster','cluster_Cat','status','category','payment_method']
st_df3 = st_df2.drop(columns_to_drop, axis=1)

In [16]:
len(st_df3)

9521

In [11]:
st_df2.T

,0,4,8,9,14,16,17,24,27,29,...,30481,30483,30486,30487,30488,30489,30491,30497,30509,30538
status,complete,complete,complete,complete,complete,complete,complete,complete,complete,complete,...,complete,complete,complete,complete,complete,complete,complete,complete,complete,complete
category,Mobiles & Tablets,Soghaat,Mobiles & Tablets,Entertainment,Appliances,Mobiles & Tablets,Men's Fashion,Women's Fashion,Appliances,Men's Fashion,...,Mobiles & Tablets,Entertainment,Beauty & Grooming,Mobiles & Tablets,Mobiles & Tablets,Mobiles & Tablets,Mobiles & Tablets,Superstore,Men's Fashion,Men's Fashion
payment_method,easypay_voucher,cod,Payaxis,Easypay_MA,Easypay,easypay_voucher,cod,cod,Easypay,easypay_voucher,...,jazzvoucher,jazzvoucher,jazzvoucher,jazzvoucher,jazzvoucher,jazzvoucher,jazzvoucher,jazzvoucher,customercredit,cod
respond_to_discount,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
total_spent,27394.19,3971.3,10711.079,18548.33,10151.17601,4821.39,5636.5,2071.11,49523.55,9270.9,...,5445.44,2185.25,623.7,5445.44,5750.654,5852.392,4389.294,746.414,9627.535,147.5
total_discount_received,3406.01,500.0,1548.421,4956.97,1880.82399,715.71,1240.0,286.69,12243.85,250.0,...,638.56,183.85,89.1,638.56,668.746,678.808,509.106,126.586,90.99,100.0
no_of_transactions,41,11,9,32,56,4,6,6,78,42,...,4,5,4,4,4,4,3,4,10,2
days_till_last_interaction,93,224,176,158,106,374,419,322,251,108,...,206,159,206,206,206,206,206,206,136,154
years_of_stay,11,14,10,8,11,10,21,4,13,18,...,6,10,6,5,6,26,21,4,27,21
age_binned_Children,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
kmeans = KMeans(n_clusters=6, init = 'k-means++', random_state = 0)

In [18]:
y_kmeans = kmeans.fit_predict(st_df3)
st_df2['cluster'] = kmeans.labels_

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-18-75a512429c11>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  st_df2['cluster'] = kmeans.labels_


In [19]:
st_df2.groupby('cluster').size()

cluster
0     624
1    6651
2      27
3    2039
4     176
5       4
dtype: int64

In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
population_mean = st_df2['total_discount_received'].mean()

def simple_random_sampling(data, sample_sizes):
        np.random.seed(42)
        samples = []
        for size,confidence_interval in sample_sizes:
            indices = random.sample(range(len(data)),size)
            srs_sample = data.iloc[indices]
            srs_mean = srs_sample['total_discount_received'].mean()
            srs_sd = np.std(srs_sample['total_discount_received'], ddof=1) 
            srs_se = srs_sd / np.sqrt(size)
            srs_absolute_error = abs(population_mean - srs_mean)
            samples.append(['Simple Random Sampling',confidence_interval, size, srs_absolute_error, srs_se])
        return samples

def stratified_sampling(data, sample_sizes):
    samples = []
    data2 = data
    data2.loc[data2['Gender_M'] == 1, 'Customer_Strata'] = 1
    data2.loc[data2['Gender_M'] != 1, 'Customer_Strata'] = 0
    
    population_mean = data['total_discount_received'].mean()
    for size,confidence_interval in sample_sizes:
        split = StratifiedShuffleSplit(n_splits=1, test_size=size)
        for x, y in split.split(data2, data2['Customer_Strata']):
            str_sample = data2.iloc[y].sort_values(by='cust_id')
        str_mean = str_sample['total_discount_received'].mean()
        str_se = np.std(str_sample['total_discount_received'], ddof=1) / np.sqrt(size)
        str_absolute_error = abs(population_mean - str_mean)
        samples.append(['Stratified Sampling',confidence_interval, size, str_absolute_error, str_se])
    return samples

def systematic_sampling(data, sample_sizes):
        np.random.seed(42)
        samples = []
        for size,confidence_interval in sample_sizes:
            step = len(data) // size
            start = np.random.randint(0, step)
            indices = np.arange(start, len(data), step = step)
            sys_sample = data.iloc[indices]
            sys_mean = sys_sample['total_discount_received'].mean()
            sys_sd = np.std(sys_sample['total_discount_received'], ddof=1) 
            sys_se = sys_sd / np.sqrt(size)
            sys_absolute_error = abs(population_mean - sys_mean)
            samples.append(['Systematic Sampling',confidence_interval, size, sys_absolute_error, sys_se])
        return samples

def cluster_sampling(data,sample_sizes):
        samples = []
        cluster1=data.loc[data['cluster'] == 0]
        cluster2=data.loc[data['cluster'] == 1]
        cluster3=data.loc[data['cluster'] == 3]


        clusters = []
        clusters.append(cluster1)
        clusters.append(cluster2)
        clusters.append(cluster3)

        random.seed(42)
        selected_cluster = random.choice(clusters)

        for size,confidence_interval in sample_sizes:
            indices = random.sample(range(len(selected_cluster)), size)
            cluster_sample = selected_cluster.iloc[indices]
            cluster_mean = cluster_sample['total_discount_received'].mean()
            cluster_se = np.std(cluster_sample['total_discount_received'], ddof=1) / np.sqrt(size)
            cluster_absolute_error = abs(population_mean - cluster_mean)
            samples.append(['Cluster Sampling',confidence_interval, size, cluster_absolute_error, cluster_se])
        return samples
    

def sampling_pipeline(data, sample_sizes):
        srs_samples = simple_random_sampling(data, sample_sizes)
        str_samples = stratified_sampling(data, sample_sizes)
        sys_samples = systematic_sampling(data, sample_sizes)
        cls_samples = cluster_sampling(data, sample_sizes)

        # Combine results into DataFrame
        train_set = srs_samples  + sys_samples + cls_samples + str_samples
        df = pd.DataFrame(train_set, columns=['Sampling Technique','Confidence Interval','Sample Size', 'Absolute Error', 'Standard Error'])

        return df


### Calculate Sample Sizes ###

In [21]:
import math

# Define the z-score values and their corresponding confidence intervals as tuples
z_scores = [(1.645, '90%'), (1.96, '95%'), (2.576, '99%')]

margin_of_error = 0.05
population_proportion = 0.5
population_size = 9521

# Calculate the sample size for each confidence interval
sample_sizes = []
for z, confidence_interval in z_scores:
    sample_size = (((z**2) * population_proportion * (1 - population_proportion)) / (margin_of_error**2)) / (1+ ((z**2) * population_proportion * (1 - population_proportion))/((margin_of_error**2)*population_size))
    sample_size = math.ceil(sample_size) # Round up to the nearest integer
    sample_sizes.append((sample_size, confidence_interval))

print("Sample sizes needed for 90%, 95%, and 99% confidence intervals:", sample_sizes)


Sample sizes needed for 90%, 95%, and 99% confidence intervals: [(264, '90%'), (370, '95%'), (621, '99%')]


In [23]:
df = st_df2
result_table = sampling_pipeline(df, sample_sizes)
print(result_table)

        Sampling Technique Confidence Interval  Sample Size  Absolute Error  \
0   Simple Random Sampling                 90%          264       61.985049   
1   Simple Random Sampling                 95%          370      175.533966   
2   Simple Random Sampling                 99%          621       33.105934   
3      Systematic Sampling                 90%          264      152.233022   
4      Systematic Sampling                 95%          370       51.971806   
5      Systematic Sampling                 99%          621       61.379790   
6         Cluster Sampling                 90%          264      801.435124   
7         Cluster Sampling                 95%          370      837.772860   
8         Cluster Sampling                 99%          621      776.133827   
9      Stratified Sampling                 90%          264      188.461193   
10     Stratified Sampling                 95%          370      102.425656   
11     Stratified Sampling                 99%      